In [61]:
from selenium import webdriver
import pandas as pd
import re

In [101]:
browser = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')

## Scrape stay at home orders

In [102]:
url = 'https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html?smid=ig-nytimes&utm_source=like2buy.curalate.com&crl8_id=3b7a7632-c666-4d1b-955d-e8cadc22aaae'
browser.get(url)

order_nodes = browser.find_elements_by_xpath("//div[starts-with(@id, 'item-')]")
print('States that have issued orders: ' + str(len(order_nodes)))

In [110]:
def get_place_name(node, type_of_node=None):
    if type_of_node == 'state':
        node = node.find_element_by_tag_name('h3')
    name = node.text
    children = node.find_elements_by_xpath('./*')
    for child in children: 
        name = name.replace(child.text, '').strip()
    return name
    
def format_order(order):
    return order.text.split(',')[0].lower()

def format_date(date):
    return re.search('[A-Z]\w* \d*', date.text)[0]

In [120]:
data = {'state': [], 'place': [], 'order': [], 'date': []}

for order_node in order_nodes:
    state_name = get_place_name(order_node, 'state')
    places = order_node.find_elements_by_class_name('l-place')
    orders = order_node.find_elements_by_class_name('l-order')
    dates = order_node.find_elements_by_class_name('l-date')                        
    
    if not places:
        data['state'].append(state_name)
        data['place'].append(None)
        data['order'].append(format_order(orders[0]))
        data['date'].append(format_date(dates[0]))
    else:
        for i, place in enumerate(places):
            data['state'].append(state_name)
            data['place'].append(get_place_name(place))
            data['order'].append(format_order(orders[i]))
            data['date'].append(format_date(dates[i]))

In [122]:
df_orders = pd.DataFrame(data)
df_orders

,state,place,order,date
0,Alabama,Birmingham,shelter in place,March 24
1,Alaska,None,stay at home,March 28
2,Arizona,None,stay at home,March 31
3,California,None,stay at home,March 19
4,Colorado,None,stay at home,March 26
...,...,...,...,...
148,Virginia,None,stay at home,March 30
149,Washington,None,stay at home,March 23
150,West Virginia,None,stay at home,March 24
151,Wisconsin,None,stay at home,March 25
